In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OrdinalEncoder, Binarizer
from imblearn.over_sampling import SMOTENC, SMOTEN, SMOTE
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from gen_ball_mix import gen_balls
from classGBTSVM import OvO_GBTSVM , MultiLabelGBTSVM, GBTSVM, OVR_GBTSVM
from classLGBTSVM import OvO_LGBTSVM, OvR_LGBTSVM, LGBTSVM
from ucimlrepo import fetch_ucirepo 
from scipy.stats import zscore

In [15]:
def evaluate_base_model(model_class, train_data, test_data, *params):
    model = model_class(*params)
    model.fit(train_data)
    y_pred = model.predict(test_data)
    accuracy = model.score(test_data)
    return accuracy

In [16]:
def evaluate_model(model_class, X_train, y_train, X_test, y_test, *params):
    model = model_class(*params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = model.score(X_test, y_test)
    return accuracy

In [17]:
ecoli = fetch_ucirepo(id=42) 
X = ecoli.data.features 
y = ecoli.data.targets.squeeze()

In [18]:
smote = SMOTE(random_state=42, k_neighbors=1)
X_res, y_res = smote.fit_resample(X, y)

In [19]:
df = pd.concat([X_res, y_res], axis=1)
target_col = y.name
enc_target = LabelEncoder()
df[target_col] = enc_target.fit_transform(df[target_col]).astype(np.float64)

In [20]:
data = df.to_numpy(dtype=np.float64)
m,n = data.shape

In [21]:
scaler = MinMaxScaler(feature_range=(-1, 1))
data[:, :-1] = scaler.fit_transform(data[:, :-1])

In [22]:
np.random.seed(0)
indices = np.random.permutation(m)
data = data[indices]
A_train=data[0:int(m*(1-0.20))]
A_test=data[int(m * (1-0.20)):]

In [23]:
pur = 1 - (0.015 * 5)                      
num = 2
A_train = gen_balls(A_train, pur=pur, delbals=num)

Radius=[]
for i in A_train:
    Radius.append(i[1])
Center=[]
for ii in A_train:
    Center.append(ii[0])
Label=[]
for iii in A_train:
    Label.append(iii[-1])
Radius=np.array(Radius)
Center=np.array(Center)
Label=np.array(Label)
Z_train=np.hstack((Center,Radius.reshape(Radius.shape[0], 1)))
Lab=Label.reshape(Label.shape[0], 1)
A_train=np.hstack((Z_train,Lab))

In [24]:
X_train = A_train[:,:-1]
X_test = A_test[:,:-1]
y_train = A_train[:,-1]
y_test = A_test[:,-1]

In [25]:
accuracy_gbtsvm = evaluate_base_model(GBTSVM, A_train, A_test, 0.1, 0.1, 0.05, 0.05)
accuracy_lgbtsvm = evaluate_base_model(LGBTSVM, A_train, A_test, 0.1, 0.1, 0.1, 0.1)
accuracy_ovo_gbtsvm = evaluate_model(OvO_GBTSVM, X_train, y_train, X_test, y_test, 0.1, 0.1, 0.05, 0.05)
accuracy_ovr_gbtsvm = evaluate_model(OVR_GBTSVM, X_train, y_train, X_test, y_test, 0.1, 0.1, 0.05, 0.05)
accuracy_multi_gbtsvm = evaluate_model(MultiLabelGBTSVM, X_train, y_train, X_test, y_test, 0.1, 0.1, 0.05, 0.05)
accuracy_ovo_lgbtsvm = evaluate_base_model(OvO_LGBTSVM, A_train, A_test, 0.1, 0.1, 0.1, 0.1)
accuracy_ovr_lgbtsvm = evaluate_base_model(OvR_LGBTSVM, A_train, A_test, 0.1, 0.1, 0.1, 0.1)

In [26]:
print(f"Accuracy of GBTSVM: {accuracy_gbtsvm}")
print(f"Accuracy of LGBTSVM: {accuracy_lgbtsvm}")
print(f"Accuracy of OvO_GBTSVM: {accuracy_ovo_gbtsvm}")
print(f"Accuracy of OVR_GBTSVM: {accuracy_ovr_gbtsvm}")
print(f"Accuracy of MultiLabelGBTSVM: {accuracy_multi_gbtsvm}")
print(f"Accuracy of OvO_LGBTSVM: {accuracy_ovo_lgbtsvm}")
print(f"Accuracy of OvR_LGBTSVM: {accuracy_ovr_lgbtsvm}")

Accuracy of GBTSVM: 0.0
Accuracy of LGBTSVM: 0.0
Accuracy of OvO_GBTSVM: 72.82608695652173
Accuracy of OVR_GBTSVM: 35.869565217391305
Accuracy of MultiLabelGBTSVM: 44.565217391304344
Accuracy of OvO_LGBTSVM: 33.69565217391305
Accuracy of OvR_LGBTSVM: 32.608695652173914
